# Цены на продукты питания

In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import plotly
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import docx
import datetime as dt
import pickle
import random
import math
import time
import os
import re
import colorsys
import arrow
import locale
import requests
import collections
import numbers
import decimal

from warnings import simplefilter

from scipy import stats
from scipy.interpolate import make_interp_spline, BSpline
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from matplotlib.lines import Line2D
import matplotlib.dates as mdates

In [2]:
path_current = os.getcwd()
path_main = os.path.sep.join(path_current.split(os.path.sep)[:-2])

In [3]:
path_current

'/Users/achasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/Мой диск/Data Science Projects/# economy-rus-dash/notebooks/data-prepare'

In [4]:
path_main

'/Users/achasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/Мой диск/Data Science Projects/# economy-rus-dash'

### To main directory

In [5]:
# change directory to main path
os.chdir(path_main)

### Load libraries, functions, palette, theme

In [6]:
# load libraries and palette
%run __theme.ipynb

In [7]:
# load functions
# from __functions import *
%run __functions.ipynb

In [8]:
# start stopwatch
start = stopwatch_start()

### Variables

In [9]:
path_img = 'docs/img/'

In [10]:
path_files = path_main + '/files'

In [11]:
path_excel_csv = path_main + '/excel-csv'

### Load Files

In [12]:
economics_data = loadit('economics_data', path=path_files, create_empty_dict=True)

In [13]:
income = economics_data['households_income'].copy()

### Back to notebook directory

In [14]:
# back to current path
os.chdir(path_current)

# Домохозяйства

In [16]:
income_plot_data = income[income['Тип'] == 'Всего'].T[1:].iloc[:, -2:].copy()

In [17]:
income_plot_data['delta'] = \
    income_plot_data.iloc[:, 0] - income_plot_data.iloc[:, 1]

In [18]:
income_plot_data['delta_town'] = \
    (income_plot_data['Домашние хозяйства, проживающиев городской местности']
     .pct_change()
     .mul(100))

income_plot_data['delta_country'] = \
    (income_plot_data['Домашние хозяйства, проживающиев сельской местности']
     .pct_change()
     .mul(100))

/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83039/3073320293.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .pct_change()
/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83039/3073320293.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .pct_change()


In [19]:
income_plot_data['delta_pct'] = \
    income_plot_data['delta_town'] - income_plot_data['delta_country']

In [20]:
income_plot_data['country/town'] = \
    (income_plot_data['Домашние хозяйства, проживающиев сельской местности']
     / income_plot_data['Домашние хозяйства, проживающиев городской местности'])

In [21]:
income_plot_data.index = income_plot_data.index.astype(int)

In [22]:
income_plot_data

,"Домашние хозяйства, проживающиев городской местности","Домашние хозяйства, проживающиев сельской местности",delta,delta_town,delta_country,delta_pct,country/town
2003,4009.0,2423.2,1585.8,NaN,NaN,NaN,0.60444
2004,5016.0,2851.3,2164.7,25.118483,17.666722,7.451762,0.568441
2005,6529.5,3604.7,2924.8,30.173445,26.423035,3.750410,0.552064
2006,7984.9,4495.4,3489.5,22.289609,24.709407,-2.419798,0.562988
2007,10354.6,5871.1,4483.5,29.677266,30.602394,-0.925128,0.567004
2008,13465.9,7786.5,5679.4,30.047515,32.624210,-2.576695,0.578238
2009,13869.4,8416.9,5452.5,2.996458,8.096064,-5.099606,0.606868
2010,16265.0,10128.7,6136.3,17.272557,20.337654,-3.065097,0.62273
2011,18291.1,11745.8,6545.3,12.456809,15.965524,-3.508715,0.642159
2012,20405.0,13320.3,7084.7,11.556987,13.404791,-1.847805,0.652796


### Save Dataset

In [23]:
economics_data['households_income_difference'] = income_plot_data

In [24]:
saveit(
    file=economics_data,
    name='economics_data',
    path=path_files
)

File 'economics_data.pkl' saved


### Execution time

In [123]:
stopwatch_stop(start)

Execution time: 0:00:01
